In [1]:
%run __init__.py

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import load, dump
import news_api
from scrapy import Scrapy
from news_api import News_api

In [3]:
newsapi_key = load("./newsapi/newsapi_key1.pkl")

In [7]:
apple_obj = News_api(newsapi_key, "2021-06-01","2021-06-07")

In [8]:
sources = ['abc-news', 'business-insider', 'financial-post', 'google-news', 'reuters',
           'nbc-news', 'techcrunch', 'wired', 'the-wall-street-journal']

In [9]:
apple_news = apple_obj.get_news(["apple"])

In [24]:
df_news = pd.DataFrame(apple_news)

In [25]:
df_news.head(3)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Daemonology.net'}",None,Daily Hacker News for 2021-06-07,The 10 highest-rated articles on\nHacker News\...,https://www.daemonology.net/hn-daily/2021-06-0...,None,2021-06-08T00:00:00Z,The 10 highest-rated articles on\r\nHacker New...
1,"{'id': None, 'name': 'Healthcare IT News'}",Rosy Matheson,Is gamification the way forward for precision ...,The #HIMSS21Europe Conference has been hearing...,https://www.healthcareitnews.com/news/emea/gam...,https://www.healthcareitnews.com/sites/hitn/fi...,2021-06-08T00:00:00Z,Singapore claims to have achieved a level of p...
2,"{'id': None, 'name': 'MobiHealthNews'}",mallory.hackett@himss.org (Mallory Hackett),New app called Voiceitt helps folks with non-s...,Voiceitt can be used conversationally or to co...,https://www.mobihealthnews.com/news/new-app-ca...,https://www.mobihealthnews.com/sites/default/f...,2021-06-08T00:00:00Z,Speech recognition company Voiceitt is making ...


In [27]:
df_news.to_csv('./data/newsapi_01jun21-07jun21.csv', index=False)

In [28]:
articles = Scrapy.scrap(df_news.url.values.tolist())

In [31]:
print('Total scraped articles:', len(articles))

Total scraped articles: 20


In [32]:
np.all(df_news.shape[0] == len(articles))

True

In [33]:
dump(articles, './data/newsapi_articles_01jun21-07jun21')

['./data/newsapi_articles_01jun21-07jun21']

In [38]:
# prepare our data concatenating the 3 main dataframe for scrubbing
df_news = pd.concat([df_news, pd.Series(articles, name='articles')],
                   axis=1,
                   ignore_index=False,
                   sort=True)

In [40]:
df_news.head(3)

,source,author,title,description,url,urlToImage,publishedAt,content,articles
0,"{'id': None, 'name': 'Daemonology.net'}",None,Daily Hacker News for 2021-06-07,The 10 highest-rated articles on\nHacker News\...,https://www.daemonology.net/hn-daily/2021-06-0...,None,2021-06-08T00:00:00Z,The 10 highest-rated articles on\r\nHacker New...,The 10 highest-rated articles on\nHacker News\...
1,"{'id': None, 'name': 'Healthcare IT News'}",Rosy Matheson,Is gamification the way forward for precision ...,The #HIMSS21Europe Conference has been hearing...,https://www.healthcareitnews.com/news/emea/gam...,https://www.healthcareitnews.com/sites/hitn/fi...,2021-06-08T00:00:00Z,Singapore claims to have achieved a level of p...,"[top left-right] Dr Charles Alessi, CCO, HIMSS..."
2,"{'id': None, 'name': 'MobiHealthNews'}",mallory.hackett@himss.org (Mallory Hackett),New app called Voiceitt helps folks with non-s...,Voiceitt can be used conversationally or to co...,https://www.mobihealthnews.com/news/new-app-ca...,https://www.mobihealthnews.com/sites/default/f...,2021-06-08T00:00:00Z,Speech recognition company Voiceitt is making ...,Photo courtesy of Voiceitt Speech recognition ...


In [41]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       20 non-null     object
 1   author       19 non-null     object
 2   title        20 non-null     object
 3   description  20 non-null     object
 4   url          20 non-null     object
 5   urlToImage   18 non-null     object
 6   publishedAt  20 non-null     object
 7   content      20 non-null     object
 8   articles     20 non-null     object
dtypes: object(9)
memory usage: 1.6+ KB


In [252]:
df_news.rename(columns={'publishedAt':'date'}, inplace=True)

In [43]:
df_news['date'] = pd.to_datetime(df_news['date'], infer_datetime_format=True)

In [44]:
df_news['source'] = df_news['source'].apply(lambda s: s['name'] if s['name']!=None else s['id'] if s['id']!=None else s)

In [47]:
df_news['word_count'] = df_news['articles'].apply(lambda x: len(x.split()))

In [48]:
df_news.head(3)

,source,author,title,description,url,urlToImage,date,content,articles,word_count
0,Daemonology.net,None,Daily Hacker News for 2021-06-07,The 10 highest-rated articles on\nHacker News\...,https://www.daemonology.net/hn-daily/2021-06-0...,None,2021-06-08,The 10 highest-rated articles on\r\nHacker New...,The 10 highest-rated articles on\nHacker News\...,30
1,Healthcare IT News,Rosy Matheson,Is gamification the way forward for precision ...,The #HIMSS21Europe Conference has been hearing...,https://www.healthcareitnews.com/news/emea/gam...,https://www.healthcareitnews.com/sites/hitn/fi...,2021-06-08,Singapore claims to have achieved a level of p...,"[top left-right] Dr Charles Alessi, CCO, HIMSS...",624
2,MobiHealthNews,mallory.hackett@himss.org (Mallory Hackett),New app called Voiceitt helps folks with non-s...,Voiceitt can be used conversationally or to co...,https://www.mobihealthnews.com/news/new-app-ca...,https://www.mobihealthnews.com/sites/default/f...,2021-06-08,Speech recognition company Voiceitt is making ...,Photo courtesy of Voiceitt Speech recognition ...,510
